# Book recommendation notebook for Model 3

This notebook contains the recommendation algorithm made for model for the book exchange app.

In [1]:
# import books dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/book_recommendations/model_2_data.csv',encoding='latin-1')

In [2]:
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
0,Z4gCnjTUahoz2qaZmk9GJt,Basics of the U.S. Health Care System: Edition 2,Nancy J. Niles,Jones & Bartlett Learning,9.781284e+12,English,3,2.3
1,PwS6wrs4jTJTRrENQ4ErfH,Financial Accounting Theory,Craig Deegan,McGraw-Hill Education Australia,9.781743e+12,English,7,4.7
2,JysYJh9MJsgB92MH86syxU,Delivering Health Care in America: A Systems A...,Leiyu Shi,Jones & Bartlett Learning,9.781284e+12,English,7,1.5
3,ZNanmaD93chZqkq5JHzM2t,Fraud and Fraud Detection: A Data Analytics Ap...,Sunder Gee,John Wiley & Sons,9.781119e+12,English,1,5.0
4,JGUfL4yHZzQhKSLPcMbc3e,Financial Institutions| Instruments and Market...,Christopher Viney,McGraw-Hill Education Australia,9.781744e+12,English,3,1.7


In [3]:
df.shape

(2151, 8)

In [6]:
# unique user id's in the dataset
df['user_id'].unique()

['Z4gCnjTUahoz2qaZmk9GJt' 'PwS6wrs4jTJTRrENQ4ErfH'
 'JysYJh9MJsgB92MH86syxU' ... 'TxfZrGa5ABCS3AEBgkwHbx'
 'dhJ4XcyfKTtYkwbrGT6QT6' 'Ke22KbhUQHPXTufLJBcSsn']


In [7]:
len(df['user_id'].unique())

2151

In [8]:
df = df.sample(frac = 1)
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
68,XBNoz3nTe39L3Vbhxs3NqC,The World According to Snoopy Volume Two,Charles M. Schulz,Open Road Media,9.781504e+12,English,9,5.0
485,PZeJnPyTUhBTGV89ckAup4,The Shadow of the Wind: The Cemetery of Forgot...,Carlos Ruiz Zafon,Hachette UK,9.780298e+12,English,7,4.4
1193,bsqcWP4cH2nJNQvzWTawMN,Christmas Ever After (Puffin Island trilogy| B...,Sarah Morgan,HarperCollins UK,9.781474e+12,English,2,4.3
1652,dijLrMBUU7DTW79Fu7Rr3V,The Memory Police,Yoko Ogawa,Random House,9.781474e+12,English,8,4.2
1551,aA2mFat4rfoZZeRQxNn5Ai,Requiem for a Dream: A Novel,Hubert Selby,Open Road Media,9.781453e+12,English,2,4.7


In [10]:
df.dtypes

user_id              object
 title               object
 author              object
 publisher           object
 isbn               float64
 language            object
 no_of_exchanges      int64
 rating             float64
dtype: object

In [24]:
df.isna().sum()

user_id               0
 title                0
 author               4
 publisher            0
 isbn               207
 language             1
 no_of_exchanges      0
 rating               0
dtype: int64

In [33]:
df.dropna(inplace=True)

In [34]:
df.isna().sum()

user_id             0
 title              0
 author             0
 publisher          0
 isbn               0
 language           0
 no_of_exchanges    0
 rating             0
dtype: int64

In [85]:
from scipy.sparse import csr_matrix
# pivot rating into movie features
mat = df.pivot(index=' isbn',columns='user_id',values=' no_of_exchanges').fillna(0)

In [86]:
mat.head()

user_id,22gThuJzsUrfdmEnA6stVe,22z5Cgp2oWgwcfXULdDMyv,237AfG45kE6tKG9KXrhmEh,25M8xKKYVweuPCMxfinYzu,25aAPXXm4cLAgwohtb7Q7F,25wgGugMawBspS8Z8BYHpK,27tCDQZw7Mpyx3RWoS6YX9,27zZfwYsi6m9EDo73XCWNn,29TKi2jiZQcAFAXpfbuC9F,2CdhCconXp93GuFFb4nq6i,2DB5qhhqW6z4KJPpkVPKiq,2Fr4kA38kNTtbMddUjZMbE,2JaepgyTUkpk8TY4PCqfq8,2KJtWmEBxRWUa9Cm6sbQBw,2KQrD6dSF3pWCa7pbENdCg,2KRm64ZTp4t57Ju7Qrx66R,2PYQD4Z32bGdanY6N6XAyv,2PkFbLhXiW3ttY3TwaGbh7,2PtuqH3TcyUjKqxmExBH7b,2QM5SU3hypCzsi9BBNpsTN,2QxRkSakxP36uxyYHDkZM4,2RbcbxzCn7RZ7asR9NsdsZ,2Rh5MibEfR8edvnKyqXUjr,2TdCP2EAqosv7Y2Zono36Y,2TwT8wT5RbdaF9qGLAc55B,2U98DYvvdAAPX7sgsz4Zdj,2UW6EnY4tfh2BKhoaYbxWP,2VsvcQR7WfNy7J8i7HmTL4,2bstVJWsXyrkbLwmoTb29W,2dCQJFTro2Da627U2CViHk,2doXTWzm5ncmM7BaRoz7Qy,2hKQzsyNMJorYa3kdeQYn2,2i9g6kN3fqUJ9yEzuh499E,2jXxzRNeLZGoD3g6G5BU5g,2jbuUvA7NxdV4ZNZQTqWFt,2ns6EUYic4Nb4BZGN94ULG,2oq5bjSR4xDW67yuhdRBrn,2pFXpMJCTAH9jPd63jix9g,2rM73GEyC9BGmDXjvic3wC,2smTV8t3HB3hRhWh2EsxdQ,...,nmicBono9shVi3aBcyd46W,nnQLLuWGJMpqkPSyRrSfQG,noAtJQouJipjzt8DPdekDe,npPeTZaZVAe7CcyrEDT4sL,npTrKwagnR8XfCPrxx3MHZ,nq8xJRNfYzn6Zzx33BYTBQ,nqSDsiLkcSfsNLLjebz2LD,nuCx2nzw3z8vGjGndxsZRR,nvUdvwHGJVSjipHnTX2D4s,nw5FAcfwxm5Th3jJyFDAkj,nwDRLW7JYf3LNTQ3s58JG9,nwqCX93dKPS4ZFoEEU54eL,nwy2cM4WXBnwDkRVebkXYE,nxHYuHwbocMYBhauoT3Wp7,nxgr27UozQyvyoZdDUTfbZ,nxqymP6iPqUP7E5m4dTR3q,o2McYrqGM7DL6EJTcUXSuc,o3UDoMFD87RMLSpdYKpPJV,o4SmZBLPWtqsZdYgpTVuYw,o6vvyvoMF2ABqwiZWnFPRM,o9iYZm5cd4KfXjwFdwW9MU,oC9XoikDeGmDopviQWQ2eG,oCYQoGDX8xJb9DGRTFJkdp,oH2rWdFEB5ph7y6Qss7hUA,oJRBzhruqz3nn2kpKXG9Xf,oK9TseNuCRDorXdysfDETa,oLkYBkdSATbpPQGAdLbZfT,oM538McsA4JkXWsKChUa4G,oMBf6ArLXDRJMeN5wfJcJs,oMkmqsyReQGwFyj2newTqt,oNmCQ9XvKwbtYgUsPGt9Yi,oP7JHFp4nb6bzyTbSMD52y,oPo8FadNFVE7NRw8vpuZyz,oPyoqoKG84pZ2csYdCNWYF,oQruRduMi4jSN52DTQNek5,oSGamAutzQYdKguVthSBvs,oWZNMmZ8mPuHwpKmrDiVw5,oXQRFWkeJfYuaYBfWB4pnT,oXtcafmg7M8EHnPtg8HXjC,oZ6gpQWsByz8gHGgS38s3C
isbn,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
mat.shape

(1544, 1939)

In [88]:
sparse_mat = csr_matrix(mat.values)

In [40]:
book_map = df[[' isbn', ' title']]

In [41]:
book_map.head()

,isbn,title
68,9.781504e+12,The World According to Snoopy Volume Two
485,9.780298e+12,The Shadow of the Wind: The Cemetery of Forgot...
1193,9.781474e+12,Christmas Ever After (Puffin Island trilogy| B...
1652,9.781474e+12,The Memory Police
1551,9.781453e+12,Requiem for a Dream: A Novel


In [43]:
book_map

,isbn,title
68,9.781504e+12,The World According to Snoopy Volume Two
485,9.780298e+12,The Shadow of the Wind: The Cemetery of Forgot...
1193,9.781474e+12,Christmas Ever After (Puffin Island trilogy| B...
1652,9.781474e+12,The Memory Police
1551,9.781453e+12,Requiem for a Dream: A Novel
...,...,...
232,9.780142e+12,The Outsider
114,9.781119e+12,The Jossey-Bass Reader on Educational Leadersh...
1005,9.780349e+12,Year One
1401,9.789569e+12,La Tregua


In [50]:
book_map.values

array([[9781504050784.0, 'The World According to Snoopy Volume Two'],
       [9780297857136.0,
        'The Shadow of the Wind: The Cemetery of Forgotten Books 1| Book 1'],
       [9781474030946.0,
        'Christmas Ever After (Puffin Island trilogy| Book 3)'],
       ...,
       [9780349414959.0, 'Year One'],
       [9789568992118.0, 'La Tregua'],
       [9780141957777.0, 'The Power of Six']], dtype=object)

In [107]:
hash_map = dict(zip(book_map[' isbn'],book_map[' title']))

In [108]:
hash_map

{9781504050784.0: 'The World According to Snoopy Volume Two',
 9780297857136.0: 'The Shadow of the Wind: The Cemetery of Forgotten Books 1| Book 1',
 9781474030946.0: 'Christmas Ever After (Puffin Island trilogy| Book 3)',
 9781473520691.0: 'The Memory Police',
 9781453239698.0: 'Requiem for a Dream: A Novel',
 9780262321860.0: 'Processing| second edition: A Programming Handbook for Visual Designers and Artists',
 9781448109265.0: "The Orphan Master's Son: Barack Obama\x92s Summer Reading Pick 2019",
 9780062369635.0: 'To Kill a Mockingbird (Enhanced Edition)',
 9780007279364.0: 'PS| I Love You: The emotional| heartbreaking romantic fiction book from the number one best selling author of Postscript',
 9780349141817.0: 'The Hundred-Year-Old Man Who Climbed Out of the Window and Disappeared',
 9789384049171.0: 'Four Idiots: \xa0story of everyone',
 9781472002815.0: "The Surgeon's Doorstep Baby (Mills & Boon Medical)",
 9781405909525.0: 'Black Wind',
 9781948175104.0: 'Alpha Dragon Awakes

In [53]:
!pip install fuzzywuzzy

In [101]:
# function for searching books in database using keywords
from fuzzywuzzy import fuzz 
def matching(book_map, fav_book):
  """
  returns the closest match via fuzzy ratio
  If no match found, returns None

  Parameters
  --------
  book_map : dataframe of book titles along with their isbn
  fav_book : name of the favourite book

  Return
  -------
  index of closest match
  """
  match = []
  # get match
  for isbn,title in book_map.items():
    ratio = fuzz.ratio(title.lower(),fav_book.lower())
    if ratio >=60:
      match.append((title, isbn, ratio))
    # sort
    match = sorted(match,key = lambda x:x[2])[::-1]
  if not match:
    print("No match found!!!")
  else:
    print("Found possible matches in our database: {0}".format([x[0] for x in match]))
    return match[0][1]

In [109]:
matching(hash_map,"Requiem for Dream")

Found possible matches in our database: ['Requiem for a Dream: A Novel']


9781453239698.0

In [92]:
# function for making recommendations
import time
from sklearn.neighbors import NearestNeighbors
def recommend(fav_book,n_recommendations,book_map,data):
  """
  Returns the top n recommendations based on the keyword entered

  Parameters
  ---------
  fav_book : Keyword for favourite book
  n_recommendations : Number of recommendations to be made
  book_map : book database with title and isbn
  data : csr matrix of user_id, ratings and isbn

  Return
  --------
  list of top n book recommendations
  """
  # instantiating model
  model = NearestNeighbors(algorithm ='brute',metric='cosine')
  # fitting the model
  model.fit(data)
  # get input book isbn
  book_isbn = matching(book_map, fav_book)
  # inference
  print("Recommendation system starting to make inference......")
  t0 = time.time()
  distances, isbns = model.kneighbors(
      data[book_isbn],
      n_neighbors=n_recommendations+1
  )
  recommends = sorted(list(zip(isbns.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  print(f"Time Taken for recommendation : {time.time()-t0}")
  return recommends 

In [67]:
book_map

,isbn,title
68,9.781504e+12,The World According to Snoopy Volume Two
485,9.780298e+12,The Shadow of the Wind: The Cemetery of Forgot...
1193,9.781474e+12,Christmas Ever After (Puffin Island trilogy| B...
1652,9.781474e+12,The Memory Police
1551,9.781453e+12,Requiem for a Dream: A Novel
...,...,...
232,9.780142e+12,The Outsider
114,9.781119e+12,The Jossey-Bass Reader on Educational Leadersh...
1005,9.780349e+12,Year One
1401,9.789569e+12,La Tregua


In [103]:
book = "The World"
recommend(book,n_recommendations=10,book_map=hash_map,data=sparse_mat)

Found possible matches in our database: ['The Lost World', 'The Wren', 'The World in Half', 'The New Girl', 'The Wolf Gift', 'The Cold Dish', 'The Goldfinch', 'The Savior', 'Fatherland', 'The Terror', "The Doctor's Lady", 'The War of the Worlds']
Recommendation system starting to make inference......


IndexError: ignored

In [78]:
book_map[book_map[' isbn']==9781407097626]

,isbn,title
852,9.781407e+12,The Lost World


In [89]:
print(sparse_mat[1])

  (0, 862)	5.0
